In [6]:
import mxnet as mx
from mxnet import autograd
from mxnet import gluon
from mxnet import image
from mxnet import init
from mxnet import nd
from mxnet.gluon.data import vision
from mxnet.gluon.model_zoo import vision as models
import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
import h5py
import os
from time import time
import matplotlib.pyplot as plt
%run ../utils.ipynb

#%matplotlib inline
%config InlineBackend.figure_format = 'retina'

ctx = [mx.gpu(i) for i in range(1)]
#ctx = mx.gpu(0)

ERROR:root:File `'../utils.ipynb.py'` not found.


In [7]:
root_dir = "/data/ai/data/kaggle_dog-breed-identification/"
data_dir = os.path.join(root_dir, "data")
features_dir = os.path.join(root_dir, "features")


In [8]:
def get_mean_std(df): 
    for i, (fname, breed) in tqdm(df.iterrows(), total=n):
        img = cv2.imread(os.path.join(data_dir, "train/%s.jpg" % fname))
        img = cv2.resize(img, (224, 224)) / 255.0
        #img = mx.image.imread(os.path.join(data_dir, "train/%s.jpg"  %fname))
        if (i == 0):
            images = np.array([img])
        else:
            images = np.append(images, [img], axis=0)
    mean = images.mean(axis=(0,1,2))
    std = images.std(axis=(0,1,2))
    return mean, std

#mean = np.array([ 0.39186783, 0.45182955, 0.47607605])
#std = np.array([ 0.26173923, 0.2573802, 0.26252426])

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

In [13]:
first = False
def load_train_data(df):
    for i, (fname, breed) in tqdm(df.iterrows(), total=n):
        img = cv2.imread(os.path.join(data_dir,'train/%s.jpg' % fname))
        img_224 = ((cv2.resize(img, (224, 224))[:,:,::-1] / 255.0 - mean) / std).transpose((2, 0, 1))
        img_299 = ((cv2.resize(img, (299, 299))[:,:,::-1] / 255.0 - mean) / std).transpose((2, 0, 1))
        
        X_224[int(i)] =  nd.array(img_224)
        X_299[int(i)] = nd.array(img_299)
        y[int(i)] = synset.index(breed)
        nd.waitall()
        
if first:
    df = pd.read_csv(os.path.join(data_dir,'labels.csv'))
    synset = sorted(set(df['breed']))
    n = len(df)
    print("no of labels", len(synset))
    print("no of train data", n)

    X_224 = nd.zeros((n, 3, 224, 224))
    X_299 = nd.zeros((n, 3, 299, 299))
    y = nd.zeros((n,))
    load_train_data(df)

In [14]:
if first:
    nd.save(os.path.join(data_dir,'train.nd'), [X_224, X_299, y])
    nd.save(os.path.join(data_dir, 'labels.nd'), y)
else: 
    X_224, X_299, y = nd.load(os.path.join(data_dir,'train.nd'))
    #y = nd.load(os.path.join(data_dir, 'labels.nd'))

In [15]:
print(X_224.shape, X_299.shape, y.shape)

(10222, 3, 224, 224) (10222, 3, 299, 299) (10222,)


In [16]:
def load_test_data():
    for i, fname in tqdm(enumerate(df_test['id']), total=n_test):
        img = cv2.imread(os.path.join(data_dir,'test/%s.jpg' % fname))
        img_224 = ((cv2.resize(img, (224, 224))[:,:,::-1] / 255.0 - mean) / std).transpose((2, 0, 1))
        img_299 = ((cv2.resize(img, (299, 299))[:,:,::-1] / 255.0 - mean) / std).transpose((2, 0, 1))
        
        X_224_test[i] = nd.array(img_224)
        X_299_test[i] = nd.array(img_299)
        
        nd.waitall()
        
if first:
    df_test = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))
    n_test = len(df_test)
    X_224_test = nd.zeros((n_test, 3, 224, 224))
    X_299_test = nd.zeros((n_test, 3, 299, 299))
    load_test_data()

In [17]:
if first:
    nd.save(os.path.join(data_dir, 'test.nd'), [X_224_test, X_299_test])
else:
    X_224_test, X_299_test = nd.load(os.path.join(data_dir, 'test.nd'))

In [18]:
print(X_224_test.shape, X_299_test.shape)

(10357, 3, 224, 224) (10357, 3, 299, 299)


In [19]:
def get_inceptionv3_features(data, ctx):
    net = models.get_model("inceptionv3", pretrained=True, ctx=ctx)
    features_net = gluon.nn.HybridSequential()
    with features_net.name_scope():
        features_net.add(net.features)
        features_net.add(net.classifier[0])
        features_net.add(net.classifier[1])
        features_net.add(net.classifier[2])
    return features_net(data)

In [20]:
def save_features(model_name, data_train_iter, data_test_iter, ignore=False):
    # 文件已存在
    if os.path.exists(os.path.join(features_dir, 'features_train_%s.nd' % model_name)) and ignore:
        if os.path.exists(os.path.join(features_dir, 'features_test_%s.nd' % model_name)):
            return
    
    #print("loading", model_name, "cpu memory", get_mem())
    net = models.get_model(model_name, pretrained=True, ctx=ctx)
    #print("loaded", model_name, "cpu memory", get_mem())
    
    for prefix, data_iter in zip(['train', 'test'], [data_train_iter, data_test_iter]):
        features = []
        i = 0
        start = time()
        for data in tqdm(data_iter):
            # 并行预测数据
            if (i == 0):
                print(model_name, prefix, "data shape", data[0].shape, data[1].shape)
            for data_slice in gluon.utils.split_and_load(data[0], ctx, even_split=False):
                if (model_name == "inceptionv3"):
                    feature = get_inceptionv3_features(data_slice, ctx) 
                else:
                    feature = net.features(data_slice)
                if (i == 0):
                    print(i, model_name, prefix, "data slice",  data_slice.shape)
                    print(i, model_name, prefix, "feature shape",  feature.shape)
                if model_name in {"alexnet", "densenet121", "densenet169"}:
                    pass
                else:
                    #if 'squeezenet' in model_name:
                    feature = gluon.nn.GlobalAvgPool2D()(feature)
                feature = gluon.nn.Flatten()(feature)
                if (i == 0):
                    print(model_name, prefix, "flatten feature shape",  feature.shape)
                features.append(feature.as_in_context(mx.cpu()))
                i += 1
            nd.waitall()
        print(model_name, prefix, "feature length", len(features), "time", time() - start)
        features = nd.concat(*features, dim=0)
        print(model_name, prefix, "final feature shape", features.shape)
        nd.save(os.path.join(root_dir, "features", 'features_%s_%s.nd') % (prefix, model_name), features)

In [ ]:
batch_size = 128

print(X_224.shape, y.shape)
print(X_299.shape, y.shape)
print(X_224_test.shape)
print(X_299_test.shape)
data_iter_224 = gluon.data.DataLoader(gluon.data.ArrayDataset(X_224, y), batch_size=batch_size, last_batch="keep")
data_iter_299 = gluon.data.DataLoader(gluon.data.ArrayDataset(X_299, y), batch_size=batch_size, last_batch="keep")

data_test_iter_224 = gluon.data.DataLoader(gluon.data.ArrayDataset(X_224_test, X_224_test), 
                                           batch_size=batch_size, last_batch="keep")
data_test_iter_299 = gluon.data.DataLoader(gluon.data.ArrayDataset(X_299_test, X_299_test), 
                                           batch_size=batch_size, last_batch="keep")


(10222, 3, 224, 224) (10222,)
(10222, 3, 299, 299) (10222,)
(10357, 3, 224, 224)
(10357, 3, 299, 299)


In [ ]:
# Sample test
import itertools
for data in itertools.islice(data_iter_224,2):
    print(data[0].shape, data[1].shape)
    for data_slice in gluon.utils.split_and_load(data[0], ctx, even_split=False):
        print("data slice", data_slice.shape)

    
data_iter_299_sample = itertools.islice(data_iter_299, 10)
data_test_iter_299_sample = itertools.islice(data_test_iter_299, 10)
data_iter_224_sample = itertools.islice(data_iter_224, 10)
data_test_iter_224_sample = itertools.islice(data_test_iter_224, 10)
#gluon.utils.split_and_load??

(128, 3, 224, 224) (128,)


In [ ]:
from mxnet.gluon.model_zoo.model_store import _model_sha1

x_n, x_test_n, _, _  = (X_299.shape[0], X_299_test.shape[0], X_224.shape[0], X_224_test.shape[0])
#x_n=x_test_n=1
print("train", x_n, "test", x_test_n)

print(_model_sha1.keys())
for model in sorted(_model_sha1.keys()):
    if model == 'inceptionv3':
        save_features(model, data_iter_299, data_test_iter_299, ignore= True)
    else:
        if model in {"vgg11", "vgg11_bn", "vgg13", "vgg13_bn", 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn', 
                     "densenet161", "densenet201" }:
            #save_features(model, data_iter_224, data_test_iter_224, ignore=True)
            pass
        else:
            save_features(model, data_iter_224, data_test_iter_224, ignore=True)

In [14]:
#get_mem()

In [11]:
data = nd.arange(18).reshape((1,2,3,3))
data1 = gluon.nn.GlobalAvgPool2D()(data)
print(data1.shape)

data2 = nd.arange(18).reshape((1,18))
data3 = gluon.nn.GlobalAvgPool2D()(data2)
print(data3.shape)

(1, 2, 1, 1)


MXNetError: [22:10:38] src/operator/./pooling-inl.h:184: Check failed: dshape.ndim() >= 3U (2 vs. 3) Pooling: Input data should be  3D in (batch, channel, x) Or 4D in (batch, channel, y, x)  Or 5D in (batch, channel, d, y, x)

Stack trace returned 10 entries:
[bt] (0) /home/ubuntu/miniconda3/envs/gluon/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x26ae5c) [0x7f1145febe5c]
[bt] (1) /home/ubuntu/miniconda3/envs/gluon/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x252b8d1) [0x7f11482ac8d1]
[bt] (2) /home/ubuntu/miniconda3/envs/gluon/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x22f80e7) [0x7f11480790e7]
[bt] (3) /home/ubuntu/miniconda3/envs/gluon/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x214dbd2) [0x7f1147ecebd2]
[bt] (4) /home/ubuntu/miniconda3/envs/gluon/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x213a149) [0x7f1147ebb149]
[bt] (5) /home/ubuntu/miniconda3/envs/gluon/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2090721) [0x7f1147e11721]
[bt] (6) /home/ubuntu/minic